In [ ]:
!pip install rake-nltk
!pip install git+https://github.com/boudinfl/pke.git
!pip install swifter
!pip install fasttext --upgrade 
!pip3 install summa
!pip3 install pandarallel

In [ ]:
from pandarallel import pandarallel
import re
import string
import sys
import html
import pandas as pd
import json
import csv
import glob
import os.path
import time
import nltk
import logging as lg
from datetime import datetime as dt, timedelta
from gensim.summarization import keywords
from random import *
import numpy as np
from gensim.utils import simple_preprocess

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pandarallel.initialize()

In [ ]:
punc = '''!()[]|{};":\<>/@#$%^&*_~'''

#Preprocess text, punctuations etc.
relp = " "*len(punc)
def preprocess(s):
  result = re.sub(r"’", "'", str(s))
  result = re.sub(r"\n", ' ', str(result))
  result = re.sub(r"r/", ' ', str(result))
  result = result.encode("ascii", "ignore").decode()
  result = re.sub(r"http\S+", ' ', str(result), flags=re.MULTILINE)
  result = html.unescape(result)
  result = result.translate(str.maketrans(punc,relp))
  result = re.sub(' +', ' ',str(result))
  result = result.strip()
  return result;

In [ ]:
#Load dataset
df = pd.read_csv("/content/drive/My Drive/datasets/reddit/raw_dataset.csv",engine="python")
df["subreddit"] = df["subreddit"].str.lower()
df["text"] = df["text"].apply(lambda x: '\n'.join(preprocess(x.splitlines())))

In [ ]:
print(df.info)
df.head()

In [ ]:
pol_arr = ["politics","politicalhumor","politicalcompassMemes","conservative",
            "therightcantmeme","neoliberal","democrats","politicaldiscussion",
            "republicans", "libertarian"]

In [ ]:
#Extract keywords using textrank implementation in gensim
def keywords_tr(x):
  kws = keywords(x,ratio=1,lemmatize=True,scores=True)[:10]
  return json.dumps(dict(kws))

In [ ]:
#TextRank is ridiculously slow, and I can't find any gpu implementations of it. 
#So processing in chunks using multiprocessing.
def split_dataframe(df, chunk_size = 5000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [ ]:
#Split and apply
i = 0
for df_chunk in split_dataframe(df,chunk_size=1000):
  print(i)
  df_chunk["text"]=df_chunk["text"].parallel_apply(lambda x: keywords_tr(x))
  df_chunk.to_csv(f"/content/drive/My Drive/datasets/reddit/keywords/{i}.csv",index=False)
  i+=1

In [ ]:
#Merge
path = r'/content/drive/My Drive/datasets/reddit/keywords/' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

In [ ]:
#Final Dataframe    
dff = pd.concat(li, axis=0, ignore_index=True)
dff["keywords"] = dff["text"].apply(lambda x: list(json.loads(x).keys()))
dff["text_rank_scores"] = dff["text"].apply(lambda x: list(json.loads(x).values()))
dff["label"]=dff["subreddit"].apply(lambda x:x in pol_arr)
dff.drop(columns=["text"],inplace=True)
dff.info()

In [ ]:
dff.to_csv("/content/drive/My Drive/datasets/reddit/extracted_keywords_freq.csv",index=False)